# Pandas
[Библиотека](https://ru.wikipedia.org/wiki/Pandas) для работы с таблицами, по сути являющейся реализацией нереляционной базы данных.

Рекомендуемое ядро для запуска кода: `Python v3.7` или выше.

In [1]:
from os import mkdir
from os.path import isdir, join as join_path
from functools import partial
from warnings import filterwarnings

import pandas as pd


filterwarnings('ignore')

DATA_DIR = 'class_data/'  # Папка, куда мы будем сохранять все файлы
if not isdir(DATA_DIR):
    mkdir(DATA_DIR)

to_data_dir = partial(join_path, DATA_DIR)
print(f"Пример работы функции 'to_data_dir': {to_data_dir('test.file')}")

Пример работы функции 'to_data_dir': class_data/test.file


Основным объектом для работы является `DataFrame` &mdash; таблица.

Простейший пример датафрейма:

In [2]:
df = pd.DataFrame(
    data={
        'Name': ('Andrew', 'Shaley', 'Kimberley'),
        'Sex':  ('m', 'f', 'f'),
        'Age': (22, 34, 18),
        'Wealth, $': (1e6, 1e5, -1200)
    }
)
df

,Name,Sex,Age,"Wealth, $"
0,Andrew,m,22,1000000.0
1,Shaley,f,34,100000.0
2,Kimberley,f,18,-1200.0


Давайте разберёмся, из каких элементов состоит этот объект и какой функционал он представляет.

Во-первых, это имена строк и столбцов.

Доступ к ним можно получить следующими командами:

In [3]:
df.index    # Для строк

RangeIndex(start=0, stop=3, step=1)

In [4]:
df.columns  # Для столбцов

Index(['Name', 'Sex', 'Age', 'Wealth, $'], dtype='object')

Как мы можем догадаться, структуры данных, хранящие имена строк и столбцов, называются `индексами`.

Что же это такое и зачем это нужно?

## pd.Index
### Преобразование имён строк и столбцов

В нашем примере имена строк соответствовали их порядковому номеру.

Но строки (как и столбцы) можно называть произвольным образом:

In [5]:
df.index = ['User_1', 'User_2', 'User_400']
df

,Name,Sex,Age,"Wealth, $"
User_1,Andrew,m,22,1000000.0
User_2,Shaley,f,34,100000.0
User_400,Kimberley,f,18,-1200.0


Индексы можно удобно преобразовывать:

In [6]:
df.index = df.index.map(lambda user_id: int(user_id.split('_')[-1]))
df

,Name,Sex,Age,"Wealth, $"
1,Andrew,m,22,1000000.0
2,Shaley,f,34,100000.0
400,Kimberley,f,18,-1200.0


Таким же образом названия колонок можно, например, заключить в кавычки:

In [7]:
df.columns = df.columns.map(lambda colname: f"'{colname}'")
df

,'Name','Sex','Age',"'Wealth, $'"
1,Andrew,m,22,1000000.0
2,Shaley,f,34,100000.0
400,Kimberley,f,18,-1200.0


И обратно:

In [8]:
df.columns = df.columns.map(lambda colname: colname.split("'")[1])
df

,Name,Sex,Age,"Wealth, $"
1,Andrew,m,22,1000000.0
2,Shaley,f,34,100000.0
400,Kimberley,f,18,-1200.0


### Множественные операции
Для различных индексов можно проивзодить простые операции, применимые к множествам. Допустим, мы имеем два датасета с разными строчными индексами, и нам нужно найти общие имена строк.

In [9]:
df_1 = pd.DataFrame(
    data={
        'Name': ('Andrew', 'Shaley', 'Kimberley'),
        'Sex':  ('m', 'f', 'f'),
        'Age': (22, 34, 18),
        'Wealth, $': (1e6, 1e5, -1200)
    },
    index=(200, 300, 400)
)
df_1

,Name,Sex,Age,"Wealth, $"
200,Andrew,m,22,1000000.0
300,Shaley,f,34,100000.0
400,Kimberley,f,18,-1200.0


In [10]:
df_2 = pd.DataFrame(
    data={
        'Name': ('Andrew', 'Shaley', 'Kimberley'),
        'Sex':  ('m', 'f', 'f'),
        'Age': (22, 34, 18),
        'Wealth, $': (1e6, 1e5, -1200)
    },
    index=(400, 300, 600)
)
df_2

,Name,Sex,Age,"Wealth, $"
400,Andrew,m,22,1000000.0
300,Shaley,f,34,100000.0
600,Kimberley,f,18,-1200.0


Сделать это можно следующим образом:

In [11]:
df_2.index.intersection(df_1.index)  # Общие имена строк: 400, 300

Int64Index([400, 300], dtype='int64')

In [12]:
df_2.index & df_1.index              # То же самое

Int64Index([400, 300], dtype='int64')

In [13]:
df_2.index.difference(df_1.index)    # Уникальные для df_2 имена строк

Int64Index([600], dtype='int64')

In [14]:
# Преобразуем имена строк обратно к виду User_ID

df.index = ['User_1', 'User_2', 'User_400']
df

,Name,Sex,Age,"Wealth, $"
User_1,Andrew,m,22,1000000.0
User_2,Shaley,f,34,100000.0
User_400,Kimberley,f,18,-1200.0


### Индексация
#### Одиночная

Можно производить выделение определённых строк из датафрейма:

In [15]:
user_1 = df.loc['User_1']
user_1

Name         Andrew
Sex               m
Age              22
Wealth, $     1e+06
Name: User_1, dtype: object

In [16]:
user_1['Wealth, $']

1000000.0

Как и выделение столбцов:

In [17]:
df['Name']

User_1         Andrew
User_2         Shaley
User_400    Kimberley
Name: Name, dtype: object

#### Множественная

Можно производить множественное выделение.

Этим свойством, например, уже не может похвастаться стандартный питоновский словарь.

In [18]:
df.loc[['User_1', 'User_400']]  # Строки

,Name,Sex,Age,"Wealth, $"
User_1,Andrew,m,22,1000000.0
User_400,Kimberley,f,18,-1200.0


In [19]:
df[['Name', 'Age']]             # Столбцы

,Name,Age
User_1,Andrew,22
User_2,Shaley,34
User_400,Kimberley,18


### Именование

Для удобства индексы можно называть своими именами

In [20]:
df.index.name = 'User ID'
df

,Name,Sex,Age,"Wealth, $"
User ID,,,,
User_1,Andrew,m,22,1000000.0
User_2,Shaley,f,34,100000.0
User_400,Kimberley,f,18,-1200.0


In [21]:
df.columns.name = 'Features'
df

Features,Name,Sex,Age,"Wealth, $"
User ID,,,,
User_1,Andrew,m,22,1000000.0
User_2,Shaley,f,34,100000.0
User_400,Kimberley,f,18,-1200.0


## pd.DataFrame

### Транспонирование

In [22]:
df.T

User ID,User_1,User_2,User_400
Features,,,
Name,Andrew,Shaley,Kimberley
Sex,m,f,f
Age,22,34,18
"Wealth, $",1e+06,100000,-1200


In [23]:
df.columns.name = None

### Конвертация в `np.ndarray`

In [24]:
df.values

array([['Andrew', 'm', 22, 1000000.0],
       ['Shaley', 'f', 34, 100000.0],
       ['Kimberley', 'f', 18, -1200.0]], dtype=object)

### Применение функций

In [25]:
df.apply(lambda row: row.values, axis=1)   # Построчно

User ID
User_1       [Andrew, m, 22, 1000000.0]
User_2        [Shaley, f, 34, 100000.0]
User_400    [Kimberley, f, 18, -1200.0]
dtype: object

In [26]:
df.apply(lambda row: set(row), axis=0)     # Поколоночно

Name            {Shaley, Andrew, Kimberley}
Sex                                  {m, f}
Age                            {34, 18, 22}
Wealth, $    {1000000.0, -1200.0, 100000.0}
dtype: object

In [27]:
df.applymap(lambda x: str(x) + ' Some Suffix')  # Поэлементно

,Name,Sex,Age,"Wealth, $"
User ID,,,,
User_1,Andrew Some Suffix,m Some Suffix,22 Some Suffix,1000000.0 Some Suffix
User_2,Shaley Some Suffix,f Some Suffix,34 Some Suffix,100000.0 Some Suffix
User_400,Kimberley Some Suffix,f Some Suffix,18 Some Suffix,-1200.0 Some Suffix


### Подсчёт статистик

`pd.DataFrame` имеет множество методов для расчёта различных статистик.

Как пример, можно посчитать среднее значение.

In [28]:
df = pd.DataFrame(
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],
    index=[100, 200, 300]
)
df

,0,1,2
100,1,2,3
200,4,5,6
300,7,8,9


In [29]:
df.mean()         # По столбцам

0    4.0
1    5.0
2    6.0
dtype: float64

In [30]:
df.mean(axis=1)   # По строкам

100    2.0
200    5.0
300    8.0
dtype: float64

In [31]:
df.mean().mean()  # По всем элементам

5.0

Но лучше использовать метод, приведённый ниже

In [32]:
df.values.mean()

5.0

### Сохранение в текстовый файл

In [33]:
df.to_csv(to_data_dir('df.tsv'), sep='\t')

In [34]:
with open(to_data_dir('df.tsv'), 'r') as out:
    print(''.join(out.readlines()))

	0	1	2
100	1	2	3
200	4	5	6
300	7	8	9



### Чтение из текстового файла

In [35]:
df = pd.read_csv(to_data_dir('df.tsv'), sep='\t', index_col=0)
df

,0,1,2
100,1,2,3
200,4,5,6
300,7,8,9


### Сохранение в бинарный формат
Преимущества:
- Значительно быстрее для чтения и записи на диск
- Обычно меньше по размеру, чем аналогичный текстовый файл

Оказываются существенными при работе с файлами крупнее десятков мегабайт.

Минус:
- Нечитаем для человека

In [36]:
df.to_pickle(to_data_dir('df.pkl'))

### Чтение бинарного формата

In [37]:
df = pd.read_pickle(to_data_dir('df.pkl'))

In [38]:
df

,0,1,2
100,1,2,3
200,4,5,6
300,7,8,9
